In [1]:
import pandas as pd
import numpy as np
df_mb=pd.read_excel(r"C:\Users\141823\OneDrive - Etex Group\Desktop\Etex AU - Finance - Finance\05. CO\02. Production\COOIS-Template\MB52.xlsx")
material_df = pd.read_excel(r"C:\Users\141823\OneDrive - Etex Group\Desktop\Etex AU - Finance - Finance\00. Master data\Material Master Data _Etex AU 5500.xlsx", sheet_name=0, header=None)
file_path=(r"C:\Users\141823\OneDrive - Etex Group\Desktop\Etex AU - Finance - Finance\05. CO\02. Production\COOIS-Template\MB52-Template.xlsx")
df_plant=pd.read_excel(file_path, sheet_name='Plant Mapping')

# Set the second row (index 1) as the column headers
material_df.columns = material_df.iloc[1]  # Set second row as header
material_df = material_df.iloc[2:].reset_index(drop=True)  # Drop first two rows

# Pick needed columns and remove duplicates
material_subset = material_df[['Material', 'P&L Classification']]
material_subset = material_subset.drop_duplicates(subset=['Material', 'P&L Classification'])
# Set data types for merge
df_mb['Plant']=df_mb['Plant'].astype(str)
df_plant['Plant']=df_plant['Plant'].astype(str)
df_mb['Material'] = df_mb['Material'].astype(str).str.strip().str.replace(r'\.0$', '', regex=True)
material_subset['Material'] = material_subset['Material'].astype(str).str.strip().str.replace(r'\.0$', '', regex=True)

#Merge with Material Masterdata for 'P&L Classification' and df_plant for 'Plant Name'
df_mb=df_mb.merge(material_subset, on='Material', how='left')
df_mb=df_mb.merge(df_plant, on='Plant', how='left')

#Move 'P&L Classification' to 3rd column and 'Plant Name' to 6th column
def move_column(df, col_name, new_position):
    cols = df.columns.tolist()
    cols.insert(new_position, cols.pop(cols.index(col_name)))
    return df[cols]

# First move 'P&L Classification' to column index 2 (i.e., column C)
df_mb = move_column(df_mb, 'P&L Classification', 2)

# Then move 'Plant Name' to column index 5 (i.e., column F)
df_mb = move_column(df_mb, 'Plant Name', 5)





In [3]:
# Load the workbook
from openpyxl import load_workbook

wb = load_workbook(file_path)
ws = wb["MB52-Python"]

# Clear old data but keep headers
ws.delete_rows(2, ws.max_row)  # Deletes from row 2 downwards (keeps headers)

# Write updated DataFrame bac to "MB52-Python" (including headers)
for c_idx, column in enumerate(df_mb.columns, start=1):
    ws.cell(row=1, column=c_idx, value=column)  # Write column headers

for r_idx, row in enumerate(df_mb.itertuples(index=False), start=2):  # Start from row 2 (below headers)
    for c_idx, value in enumerate(row, start=1):
        ws.cell(row=r_idx, column=c_idx, value=value)

# Auto-adjust column widths
for col in ws.columns:
    max_length = 0
    col_letter = col[0].column_letter  # Get column letter (A, B, C, etc.)
    for cell in col:
        try:
            if cell.value:
                max_length = max(max_length, len(str(cell.value)))
        except:
            pass
    ws.column_dimensions[col_letter].width = max_length + 2  # Add padding

# Save the workbook
wb.save(file_path)



17/07/2025: I have created MB52 file to merge MB52 data with Material MasterData, and Plant Mapping
The data will be written in MB52-Template sheet
In 'Block Stock TBS and MTO Report-Template', I have created two VBAs, with one to run this code, and one to import data from MB52-Template file.
# Remind end user that there will be NAN or Blank cells in the 'P&L Classification' columns and it is for the end user to go review and update the Material MasterData.